## Analiza wskaźnika Beta dla spółek z indeksu WIG20

In [1]:
#Implementacja bibliotek wykorzystywanych do analizy

import pandas as pd
import yfinance as yf
import numpy as np
import plotly.express as px

### Pobieranie potrzebnych danych (notowania są pobierane do 26.03.2025 r.)

In [2]:
#-------------------------------------------------------------------------------------

end = "2025-04-05"

#Pobieranie danych z indeksu WIG20 oraz wyciągnięcie z nich tickerów
wig_20_tickers = []
df = pd.read_html("https://www.biznesradar.pl/gielda/indeks:WIG20", thousands=None)[0]
df_tickers = df["Profil"].str.split("(").str[0]
df_tickers = df_tickers.str.split(" ").str[0]

#Dopisanie do każdego tickera końcówki ".WA", aby poprawnie odczytać dane z yfinance
for company in df_tickers:
    ticker = company + ".WA"
    wig_20_tickers.append(ticker)

#Pobieranie danych historycznych dla spółek z WIG20
df_wig20 = yf.download(wig_20_tickers, end=end)["Close"]

#-------------------------------------------------------------------------------------

#Pobieranie danych z pliku csv dla indeksu WIG i ustawienie dat jako indeksy
df_wig = pd.read_csv("wig_d.csv")
df_wig["Data"] = pd.to_datetime(df_wig["Data"])
df_wig.set_index("Data", inplace=True)

#-------------------------------------------------------------------------------------

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  20 of 20 completed


In [3]:
#Obliczenie stóp zwrotu dla analizowanych indeksów

df_wig20_zwroty = df_wig20.pct_change().dropna()
df_wig_zwroty = df_wig["Zamkniecie"].pct_change().dropna()

In [4]:
#Skrócenie danych do 60 notowań wstecz

df_wig20_zwroty_60 = df_wig20_zwroty.tail(60)
df_wig_zwroty_60 = df_wig_zwroty.tail(60)

### Porównanie stóp zwrotu spółek WIG20 ze stopami zwrotu indeksu WIG

#### Wzór na betę wykorzystywany do analizy
$$
\beta = \frac{\text{Cov}(R_i, R_m)}{\text{Var}(R_m)}
$$

Gdzie:  
- R_i – zwroty instrumentu (np. spółki)  
- R_m – zwroty rynku (benchmarku)  
- text{Cov}(R_i, R_m) – kowariancja zwrotów instrumentu i rynku  
- text{Var}(R_m)– wariancja zwrotów rynku

In [5]:
#Obliczanie bety dla każdej spółki w oparciu o benchmark WIG

beta_spolki_wig20_wig = []

for company in wig_20_tickers:

    kowariancja = df_wig20_zwroty_60[company].cov(df_wig_zwroty_60)
    wariancja = df_wig_zwroty_60.var()
    beta = kowariancja/wariancja
    beta_spolki_wig20_wig.append(beta)

In [6]:
#Wyliczanie korelacji oraz współczynnika determinacji dla spółek
 
korelacja_wig = []
R2_wig = []

for company in wig_20_tickers:
    korelacja_macierz = np.corrcoef(df_wig20_zwroty_60[company], df_wig_zwroty_60)
    korelacja = korelacja_macierz[0,1] 
    R_sq = korelacja**2
    korelacja_wig.append(korelacja)
    R2_wig.append(R_sq)

### Wyświetlenie bety, korelacji oraz wsp. determinacji dla spółek

In [7]:
#Utworzenie tabeli z powstałych wyników

tabela_beta = pd.DataFrame({
    "Spółka": wig_20_tickers,
    "Beta": beta_spolki_wig20_wig,
    "Korelacja (r)": korelacja_wig,
    "Wsp. determinacji (R2)": R2_wig,
})

In [8]:
#Wyświetlenie tabeli z wynikami

podsumowanie = tabela_beta.set_index("Spółka")
podsumowanie

,Beta,Korelacja (r),Wsp. determinacji (R2)
Spółka,,,
ALE.WA,0.981527,0.558435,0.311850
ALR.WA,1.646571,0.878586,0.771914
BDX.WA,0.905400,0.543244,0.295114
CCC.WA,0.719817,0.350219,0.122653
CDR.WA,0.390839,0.261117,0.068182
DNP.WA,0.561194,0.470231,0.221117
KGH.WA,1.384436,0.805402,0.648673
KRU.WA,0.787889,0.645911,0.417201
KTY.WA,0.863566,0.771216,0.594774


In [9]:
#Wyświetlenie wartości beta w stosunku do WIG na wykresie słupkowym (skrypt częściowo wykonany przy pomocy AI)

tabela_beta["Kolor"] = tabela_beta["Beta"].apply(lambda x: "Beta > 1" if x > 1 else "Beta <= 1")

fig = px.bar(tabela_beta.sort_values(by="Beta", ascending=False), x="Spółka", y="Beta", color="Kolor", color_discrete_map={
                 "Beta > 1": "#0f8e45",
                 "Beta <= 1": "#00FF99"
             })
fig.add_hline(y=1, line_dash="dash", line_color="black", annotation_text="Beta=1")
fig.update_layout(
    title="Wartość współczynnika Beta dla spółek z indeksu WIG20",
    title_font_size=25,
    xaxis_title="Spółka",
    yaxis_title="Wartość Beta",
    height=600
)
fig.show()

### Spółki z indeksu WIG20 (stan na 4 kwietnia 2025)

| Ticker | Spółka                  |
|--------|-------------------------|
| ALR    | Alior Bank              |
| ALE    | Allegro                 |
| BDX    | Budimex                 |
| CCC    | CCC                     |
| CDR    | CD Projekt              |
| DNP    | Dino Polska             |
| KGH    | KGHM                    |
| KRU    | Kruk                    |
| LPP    | LPP                     |
| MBK    | mBank                   |
| OPL    | Orange Polska           |
| PEO    | Pekao                   |
| PCO    | Pepco                   |
| PGE    | PGE                     |
| PKN    | PKN Orlen               |
| PKO    | PKO BP                  |
| PZU    | PZU                     |
| SPL    | Santander Bank Polska   |
| KTY    | Grupa Kęty              |
| ZAB    | Żabka                   |
